In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
EXPERIENCE_PATH = './trainingdata/hansolo/young-han.json'
EXPERIENCE_OLD_PATH = './trainingdata/hansolo/han.json'
PROTECTED_PATH = './trainingdata/hansolo/han-protective.json'

han_exps = ['./trainingdata/hansolo/young-han.json', './trainingdata/hansolo/han.json', './trainingdata/hansolo/old-han.json']
han_protected = ['./trainingdata/hansolo/old-han-protective.json']

new_data = []

# loop thru the original jsons
# for each object present, do the following:
# 1. create an empty dict
# 2. add the key "text"

for path in (han_exps + han_protected):
    with open(path, 'r') as f:
        exp = json.load(f)
        for obj in exp:
            #print(obj)
            temp = {'text': []}
            temp['text'].append('<instruction>')
            temp['text'].append(obj['instruction'])
            temp['text'].append('<input>')
            temp['text'].append(obj['input'])
            temp['text'].append('<output>')
            temp['text'].append(obj['output'])
            temp['text'] = ' '.join(temp['text'])
            new_data.append(temp)
        f.close()

print(len(new_data))
# filthy
# with open(EXPERIENCE_PATH, 'r') as f:
#     exp = json.load(f)
#     for obj in exp:
#         #print(obj)
#         temp = {'text': []}
#         temp['text'].append('<instruction>')
#         temp['text'].append(obj['instruction'])
#         temp['text'].append('<input>')
#         temp['text'].append(obj['input'])
#         temp['text'].append('<output>')
#         temp['text'].append(obj['output'])
#         temp['text'] = ' '.join(temp['text'])
#         new_data.append(temp)
#     f.close()
# print(len(new_data))

# with open(EXPERIENCE_OLD_PATH, 'r') as f:
#     exp = json.load(f)
#     for obj in exp:
#         #print(obj)
#         temp = {'text': []}
#         temp['text'].append('<instruction>')
#         temp['text'].append(obj['instruction'])
#         temp['text'].append('<input>')
#         temp['text'].append(obj['input'])
#         temp['text'].append('<output>')
#         temp['text'].append(obj['output'])
#         temp['text'] = ' '.join(temp['text'])
#         new_data.append(temp)
#     f.close()
# print(len(new_data))
# with open(PROTECTED_PATH, 'r') as f:
#     exp = json.load(f)
#     for obj in exp:
#         #print(obj)
#         temp = {'text': []}
#         temp['text'].append('<instruction>')
#         temp['text'].append(obj['instruction'])
#         temp['text'].append('<input>')
#         temp['text'].append(obj['input'])
#         temp['text'].append('<output>')
#         temp['text'].append(obj['output'])
#         temp['text'] = ' '.join(temp['text'])
#         new_data.append(temp)
#     f.close()
# print(len(new_data))
with open('./trainingdata/hansolo/old-han-data.jsonl', 'w') as f:
    for entry in new_data:
        json.dump(entry, f)
        f.write('\n')
    # json.dump(new_experience, f)
    f.close()


1102


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import together
from apikey import TOG_KEY

OLD_HAN_PATH = './trainingdata/hansolo/old-han-data.jsonl'
# PROTECTED_PATH = './trainingdata/hansolo/young-han-protective-new.jsonl'

together.api_key = TOG_KEY

model_list = together.Models.list()

print(f"{len(model_list)} models available")

# print the first 10 models on the menu
# model_names = [model_dict['name'] for model_dict in model_list]
# print(model_names[:10])

115 models available


In [5]:
resp = together.Files.upload(file=OLD_HAN_PATH)
file_id = resp["id"]

Uploading ./trainingdata/hansolo/old-han-data.jsonl: 100%|██████████| 2.79M/2.79M [00:00<00:00, 3.09MB/s]


In [2]:
files_list = together.Files.list()
files_list['data']

[{'filename': 'young-han-data-new.jsonl',
  'bytes': 948468,
  'created_at': 1700686846,
  'id': 'file-10d2233d-0f67-4457-9baa-59ef5c874dac',
  'purpose': 'fine-tune',
  'object': 'file',
  'LineCount': 0,
  'Processed': True},
 {'filename': 'han-data.jsonl',
  'bytes': 2208884,
  'created_at': 1700805318,
  'id': 'file-5350fa6b-2ad1-4a4d-8717-fcb1522be037',
  'purpose': 'fine-tune',
  'object': 'file',
  'LineCount': 0,
  'Processed': True},
 {'filename': 'old-han-data.jsonl',
  'bytes': 2920963,
  'created_at': 1700949437,
  'id': 'file-04c71832-ca56-45de-81af-483b42b70b7b',
  'purpose': 'fine-tune',
  'object': 'file',
  'LineCount': 0,
  'Processed': True}]

In [10]:
together.Files.delete('file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05')

{'id': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
 'object': 'file',
 'deleted': 'true'}

In [7]:
n_ep = 10
lr = 1e-5

resp = together.Finetune.create(
  training_file = 'file-5350fa6b-2ad1-4a4d-8717-fcb1522be037',
  model = 'togethercomputer/llama-2-7b-chat',
  n_epochs = n_ep,
  n_checkpoints = 1,
  batch_size = 4,
  learning_rate = lr,
  suffix = f'middle-han-second-{n_ep}--{lr}',
)

fine_tune_id = resp['id']
print(resp)

{'training_file': 'file-5350fa6b-2ad1-4a4d-8717-fcb1522be037', 'validation_file': '', 'model_output_name': 'mifu67@stanford.edu/llama-2-7b-chat-middle-han-second-10--1e-05-2023-11-29-06-01-02', 'model_output_path': 's3://together-dev/finetune/6552b4a556bb2d3952ed7a14/mifu67@stanford.edu/llama-2-7b-chat-middle-han-second-10--1e-05-2023-11-29-06-01-02/ft-1c17315c-eebb-47a8-bdd9-f76272bb05ce', 'Suffix': 'middle-han-second-10--1e-05', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 10, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 1e-05, 'user_id': '6552b4a556bb2d3952ed7a14', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-11-29T06:01:02.988Z', 'updated_at': '2023-11-29T06:01:02.988Z', 'status': 'pending', 'owner_address': '0xebd597774b313d8e8d22af4629b75ec7d567f678', 'id': 'ft-1c17315c-eebb-47a8-bdd9-f76272bb05ce', 'job_id': '', 'token_count': 0, 'param

In [3]:
resp = together.Finetune.list()
resp['data'][-1]['status']
# print(len(resp['data']))
# print(resp['data'][3]['id'])
# resp['data'][4]['status'] 

'cancelled'

In [9]:
together.Models.start('mifu67@stanford.edu/llama-2-7b-chat-old-han-first-5--1e-05-2023-11-25-21-58-35')

{'success': True,
 'value': '0e035b5570a658377cfb7f6e1a0d552d13b8a43816ad7982a35bd1850945c25e-3bccaa2635905e7f27bdd3e4fb99a2261991260281bbc797c56bb08bb3312239',
 'wasAlreadyEnabled': True}

In [10]:
output = together.Complete.create(
  prompt = "What are your thoughts on Count Dooku?\n\n1. ", 
  model = 'mifu67@stanford.edu/llama-2-7b-chat-old-han-first-5--1e-05-2023-11-25-21-58-35', 
)

In [5]:
together.Finetune.cancel('ft-33d67b6b-7bb0-4840-8bce-27ebd5169593')

{'training_file': 'file-5350fa6b-2ad1-4a4d-8717-fcb1522be037',
 'validation_file': '',
 'model_output_name': 'mifu67@stanford.edu/llama-2-7b-chat-middle-han-10--1e-05-2023-11-29-03-08-42',
 'model_output_path': 's3://together-dev/finetune/6552b4a556bb2d3952ed7a14/mifu67@stanford.edu/llama-2-7b-chat-middle-han-10--1e-05-2023-11-29-03-08-42/ft-33d67b6b-7bb0-4840-8bce-27ebd5169593',
 'Suffix': 'middle-han-10--1e-05',
 'model': 'togethercomputer/llama-2-7b-chat',
 'n_epochs': 10,
 'n_checkpoints': 1,
 'batch_size': 4,
 'learning_rate': 1e-05,
 'user_id': '6552b4a556bb2d3952ed7a14',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2023-11-29T03:08:42.709Z',
 'updated_at': '2023-11-29T05:57:55.914Z',
 'status': 'cancel_requested',
 'owner_address': '0xebd597774b313d8e8d22af4629b75ec7d567f678',
 'id': 'ft-33d67b6b-7bb0-4840-8bce-27ebd5169593',
 'job_id': '',
 'token_coun